## 1. Trivia Crack
<p><a href="https://www.facebook.com/TriviaCrack/">Trivia Crack</a> is a very popular mobile quiz game developed by Etermax. It's a competitive trivia game where the user can challenge other players on a variety of interesting topics answering multiple-choice questions. Check out this short trailer:</p>
<p><a href="https://youtu.be/TfStU4RWDFg"><img src="https://i.ytimg.com/vi/TfStU4RWDFg/maxresdefault.jpg" style="width: 500px; float:center"></a></p>
<p>As players progress through the game, they will win gold coins when they beat their rival. This type of currency that is generally earned by playing or just waiting is called soft currency and can be spent for in-game benefits like lives, power-ups and gems. These currencies can be used to separate some content for paying and non-paying players or it can be used to have a better understanding of the game economy and detect opportunities to maximize revenue. For example, by having special offers in some weeks of the year that might be more profitable.
<p><img src="https://149493502.v2.pressablecdn.com/wp-content/uploads/2015/06/trivia-crack-upgrades.jpg" alt></p>
<p>When the developers should release a special offer? We will explore a dataset of transactions and try to find an answer on how to put a good deal for players!</p>

In [2]:
# Importing pandas
import pandas as pd

# Reading in the data
df = pd.read_csv('./datasets/trivia_crack.csv', parse_dates=[1,3]).dropna()

# Showing the first few rows
df.head()

,user_id,user_creation_time,platform,event_time,sink_channel,amount_spent,coins_balance
0,c4e2364,2018-07-01 00:50:46.733000+00:00,Android,2018-07-01 22:39:18.613000+00:00,sink_1,480.0,14305.0
1,8738ae95,2018-07-11 22:01:00.478000+00:00,Android,2018-07-12 23:44:29.592000+00:00,sink_1,0.0,580.0
2,8738ae95,2018-07-11 22:01:00.478000+00:00,Android,2018-07-24 18:38:39.918000+00:00,sink_1,240.0,3780.0
3,d21dd44d,2018-07-15 00:57:53.116000+00:00,Android,2018-07-17 02:12:24.354000+00:00,sink_1,480.0,36200.0
4,d21dd44d,2018-07-15 00:57:53.116000+00:00,Android,2018-07-15 23:00:04.470000+00:00,sink_1,480.0,42310.0


## 2. Total amount of coins spent
<p><img src="https://i.imgur.com/MhzF4j2.png" style="width:180px; float:right"> </p>
<p>The data we have is from 95,388 players that made a transaction during July 2018.<br>The variables are:</p>
<ul>
<li><code>user_id</code> - anonymized user identifier.</li>
<li><code>user_creation_time</code> - date and time the user registered (UTC).</li>
<li><code>platform</code> - user platform. It can be Android or iOS.</li>
<li><code>event_time</code> - date and time the coin spend event was recorded (UTC).</li>
<li><code>sink_channel</code> - type of good or benefit obtained in exchange for coins.</li>
<li><code>amount_spent</code> - amount of coins spent.</li>
<li><code>coins_balance</code> - 
balance in user currencies, after the expense has been made.</li>
</ul>
<p>Let's see how much coins where spent during this period of time.</p>

In [3]:
print("Number of players:")
print(df['user_id'].nunique())

print("\nPeriod for which we have data:")
print([df['event_time'].min().strftime('%m/%d/%Y'),df['event_time'].max().strftime('%m/%d/%Y')])

print("\nTotal amount of coins spent:")
print(df['amount_spent'].sum().astype(int))

Number of players:
95388

Period for which we have data:
['07/01/2018', '07/28/2018']

Total amount of coins spent:
891244675


## 3. Percentage of total spent by platform

<p>It looks like there is a lot of coins that has been traded on July.</p>
<p>Just for fun, let's take a look on how the amount varies depending on the platform the games is played. This can be useful to detect if you should create a campaign for a specific platform.</p>

In [4]:
# Importing altair for visualizations
import altair as alt

# Adding up the total amount spent for each platform
plot_df = df.groupby('platform')['amount_spent'].sum().reset_index()

# Plotting the percentage of total amount spent by platform
alt.Chart(plot_df).transform_joinaggregate(
    total_amount='sum(amount_spent)',
).transform_calculate(
    percent_of_total="datum.amount_spent / datum.total_amount"
).mark_bar(color="#C69320").encode(
    alt.X('percent_of_total:Q', axis=alt.Axis(format='.0%'), title='percentage of total'),
    y='platform:N'
)

alt.Chart(...)

## 4. Amount spent by benefits
<p><img src="https://i.imgur.com/BNlszql.jpeg" style="width:125px; float:right"> </p>
<p>Another interesting topic to analyze is the type of benefits/goods that are bought. This can trigger more questions like why players preferred one item over another or which items we can include in a special pack promo.</p>

In [5]:
# Adding up the amount spent each day by every type of benefit
benefit_spending = df.groupby(by=[df['event_time'].dt.date,'sink_channel'])['amount_spent'].sum().reset_index()
benefit_spending['event_time'] = benefit_spending ['event_time'].astype('datetime64')

# Plotting the amount spent by type of benefit
alt.Chart(benefit_spending).mark_bar().encode(
    x='event_time',
    y='amount_spent',
    color='sink_channel'
)

alt.Chart(...)

## 5. Amount spent by day

<p><img src="https://i.imgur.com/aYpOfR5.jpeg" style="width:125px; float:right"> </p>
<p>Analysing transactions by day can enable more precise targeting of in-app offers. For example, to schedule promotions on a specific day when users are more likely to make a purchase.</p>

In [6]:
# Adding up the amount spent for each date 
difficulty = df.groupby(df['event_time'].dt.date)['amount_spent'].sum().reset_index()
difficulty['event_time'] = difficulty['event_time'].astype('datetime64')

# Plotting the total amount spent by day
base = alt.Chart(difficulty).transform_calculate(
    week_number="floor((date(datum.event_time)-1) / 7)"
).encode(
    alt.X('day(event_time):O', title='Week day'),
    alt.Y('week_number:O', axis=None),
).properties(width=600,height=150)

# Configure heatmap
heatmap = base.mark_rect(color='red').encode(
    color=alt.Color('amount_spent:Q', scale=alt.Scale(scheme="goldred"))
)

# Configure text
text = base.mark_text().encode(
    text='date(event_time):T',
    color=alt.condition(
        alt.datum.amount_spent < 20000000,
        alt.value('black'),
        alt.value('white')
    )
)

# Draw the chart
heatmap + text

alt.LayerChart(...)

## 6. Amount spent vs users registered

<p>So, during the third week of July we can notice a significant increase of coins spent. Now, let's take a closer look and compare the amount spent average vs users registered average to see if we can find some relation between the two variables.</p>

In [7]:
# Counting unique users registered for each date 
source = df.groupby([df['user_creation_time'].dt.date])['user_id'].nunique().reset_index(name = 'users_registered')
# Adding up the amount spent for each date 
source['amount_spent'] = df.groupby(df['event_time'].dt.date)['amount_spent'].sum().reset_index()['amount_spent']
source['user_creation_time'] = source['user_creation_time'].astype('datetime64')

# Plotting amount spent avg vs users registered avg
base = alt.Chart(source).encode(
    x=alt.X('user_creation_time:T', title='date')
)

# Configure line chart of amount spent avg
amount_spent = base.mark_line(color="#C69320").transform_window(
    # The field to average
    rolling_mean='mean(amount_spent)',
    # The number of values before and after the current value to include.
    frame=[-2, 0]
).encode(
    y=alt.Y('rolling_mean:Q', axis=alt.Axis(format='$f', title='amount spent avg', titleColor='#C69320'))
)

# Configure line chart of users registered avg
users_registered = base.mark_line(color='#80CEE1').transform_window(
    rolling_mean='mean(users_registered)',
    frame=[-2, 0]
).encode(
    y=alt.Y('rolling_mean:Q', axis=alt.Axis(format='d', title='users registered avg', titleColor='#80CEE1'))
)

alt.layer(amount_spent, users_registered).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

## 7. Days passed between registration and first transaction

<p>There is a similarity between the distribution of amount and users registered. The peak of coins spent can be related to an increase of users registered during the third week. Let's plot how many days pass between the registration and the first transaction.</p>

In [8]:
# Filtering the first transaction of each user
transactions = df.loc[df.groupby('user_id')['event_time'].idxmin()]
# Counting the days between the registration and the first transaction 
transactions['days'] = (df['event_time'] - df['user_creation_time']).dt.days
# Counting the amount of transactions grouped by the days passed
transactions = transactions.groupby('days')['user_id'].count().reset_index(name = 'first_transaction')

# Plotting the amount of first transactions by the days passed
alt.Chart(transactions).transform_filter(
    'datum.days > 0'
).mark_bar().encode(
    x='days',
    y='first_transaction'
)

alt.Chart(...)

## 8. Conclusion
<p><img src="https://www.triviacrack.com/static/media/willy.c2defd328754a06192ff.png" style="width:150px; float:right"> </p>
<p>It seems that the increase in money spent in the third week is due to a rise in registered users during that week. </p>
<p>New users tend to spend the coins they earned on their first day, so we can offer a good deal for new users to get acquainted with in-app purchase promotions.</p>

In [9]:
# Should we create a sweet deal on the third week of the month for new users ?
create_deal = True